# Advent of Code 2025
## Day 8
### Part 1
We create a distance matrix, then go from the shortest distance one at a time. At each step we check whether the next pair is
- completely part of an already existing circuit
- has elements part of an alredy existing circuit but not all -> insert into that
- has completely new elements -> create a new circuit

After each iteration I consolidate the lists if any overlaps occured

In [12]:
import numpy as np
def consolidate_connections(conn_list):
    """Merge any overlapping lists"""
    changed = True
    while changed:
        changed = False
        for i in range(len(conn_list)):
            for j in range(i + 1, len(conn_list)):
                set_i = set(conn_list[i])
                set_j = set(conn_list[j])
                if not set_i.isdisjoint(set_j):
                    conn_list[i] = list(set_i.union(set_j))
                    conn_list.pop(j)
                    changed = True
                    break
            if changed:
                break
    return conn_list

def find_k_smallest_unique_dist(distances, k):
    rows_ut, cols_ut = np.triu_indices_from(distances, k=1)
    unique_distances_subset = distances[rows_ut, cols_ut]
    sorted_indices_of_subset = np.argsort(unique_distances_subset)
    
    if k > len(sorted_indices_of_subset) or k < 1:
        raise IndexError(f"k={k} is out of range for the number of unique distances ({len(sorted_indices_of_subset)})")

    kth_position = k - 1
    
    row = rows_ut[sorted_indices_of_subset[kth_position]]
    col = cols_ut[sorted_indices_of_subset[kth_position]]

    return row, col

def part1(distances, max_num_conn):

    connections = []
    row, col = find_k_smallest_unique_dist(distances, 1)
    connections.append([row, col])
    k = 2
    num_conn = 1
    while num_conn <= max_num_conn-1:
        row, col = find_k_smallest_unique_dist(distances, k)
        #print(num_conn, row, col)
        elements_to_add = {row, col}
        merged = False

        for i, inner_list in enumerate(connections):
            current_set = set(inner_list)
            has_overlap = not elements_to_add.isdisjoint(current_set)
            contains_all_new = elements_to_add.issubset(current_set)

            if contains_all_new:
                merged = True
                break

            if has_overlap and not contains_all_new:
                connections[i] = list(current_set.union(elements_to_add))
                merged = True
                break
        if not merged:
            connections.append(list(elements_to_add))    

        connections = consolidate_connections(connections)

        k += 1
        num_conn += 1
    
      
    lengths_list = sorted([len(sublist) for sublist in connections], reverse=True)
    return(lengths_list[0]*lengths_list[1]*lengths_list[2])
    return connections

In [20]:
coordinates = np.loadtxt('day8.txt', dtype=int, delimiter=",")

distances = np.zeros((coordinates.shape[0], coordinates.shape[0]))

for i in range(coordinates.shape[0]):
    for j in range(coordinates.shape[0]):
        coord1 = (coordinates[j][0] - coordinates[i][0])**2
        coord2 = (coordinates[j][1] - coordinates[i][1])**2
        coord3 = (coordinates[j][2] - coordinates[i][2])**2
        distances[i,j] = np.sqrt(coord1 + coord2 + coord3)

np.fill_diagonal(distances, np.inf)
part1(distances, 1000)

83520

### Part 3
We iterate the part1 solution until there is only one list

In [17]:
def part2(distances, coordinates):

    connections = []
    row, col = find_k_smallest_unique_dist(distances, 1)
    connections.append([row, col])
    k = 2
    num_conn = 1
    new_connections_possible = True

    while new_connections_possible:
        row, col = find_k_smallest_unique_dist(distances, k)
        #print(num_conn, row, col)
        elements_to_add = {row, col}
        merged = False

        for i, inner_list in enumerate(connections):
            current_set = set(inner_list)
            has_overlap = not elements_to_add.isdisjoint(current_set)
            contains_all_new = elements_to_add.issubset(current_set)

            if contains_all_new:
                merged = True
                break

            if has_overlap and not contains_all_new:
                connections[i] = list(current_set.union(elements_to_add))
                merged = True
                break
        if not merged:
            connections.append(list(elements_to_add))    

        connections = consolidate_connections(connections)

        k += 1
        if k>= 5 and len(connections) == 1 and len(connections[0]) == len(coordinates):
            return coordinates[row][0] * coordinates[col][0]
            break
    

In [19]:
coordinates = np.loadtxt('day8.txt', dtype=int, delimiter=",")

distances = np.zeros((coordinates.shape[0], coordinates.shape[0]))

for i in range(coordinates.shape[0]):
    for j in range(coordinates.shape[0]):
        coord1 = (coordinates[j][0] - coordinates[i][0])**2
        coord2 = (coordinates[j][1] - coordinates[i][1])**2
        coord3 = (coordinates[j][2] - coordinates[i][2])**2
        distances[i,j] = np.sqrt(coord1 + coord2 + coord3)

np.fill_diagonal(distances, np.inf)
part2(distances, coordinates)

np.int64(1131823407)